In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [11]:
# Loading data
train = pd.read_csv('../../data/class_regression/prepocessed_train_data.csv', index_col='id')
test = pd.read_csv('../../data/class_regression/prepocessed_test_data.csv', index_col='id')

train = train.drop(train['playtime_forever'].nlargest(3).index)

X1_train = train.loc[train['playtime_forever'] > 0, : ]
y1_train = X1_train['playtime_forever']
X1_train = X1_train.drop(columns=['playtime_forever', 'is_play'])
X2_train = train.drop(columns=['playtime_forever', 'is_play'])
y2_train = train['is_play']

In [12]:
nnc = MLPClassifier()
nnc.fit(X2_train, y2_train)
nnr = MLPRegressor()
nnr.fit(X1_train, y1_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [13]:
cross_val_score(nnc, X2_train, y2_train, cv=10).mean()

0.642390289449113

In [14]:
cross_val_score(nnr, X1_train, y1_train, cv=10).mean()

-0.6773610247007954

In [15]:
is_play = nnc.predict(test)
playtime_forever = nnr.predict(test)
submission = pd.DataFrame(data={
    'playtime_forever': playtime_forever,
    'is_play': is_play
})
submission.loc[submission['is_play'] == 0, 'playtime_forever'] = 0
submission.loc[submission['playtime_forever'] < 0, 'playtime_forever'] = 0
submission = submission.drop(columns=['is_play'])
submission.index.name = 'id'
submission.to_csv('../../submission/class_regression/nn.csv')